In [2]:
# For reading, visualizing, and preprocessing data
import numpy as np
import pandas as pd
import seaborn as sns
import itertools
import time
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn import model_selection
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, matthews_corrcoef, f1_score, precision_score, recall_score, cohen_kappa_score, log_loss, roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# Classifiers
from sklearn.svm import NuSVC, SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier
# from tensorflow.keras.models import Sequential
#from mlxtend.classifier import StackingCVClassifier 

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer
from tensorflow.keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier

from tensorflow.keras.layers import Dropout, Activation

from sklearn.preprocessing import Normalizer
# Used to ignore warnings generated from StackingCVClassifier
import warnings
warnings.simplefilter('ignore')

In [14]:
data = pd.read_csv("dataset_final.csv")
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)


In [15]:
numeric_data = data.copy()
numeric_data.drop(columns=numeric_data.columns[0],axis=1,inplace=True)
# categorical = pd.get_dummies(numeric_data['label'])
# categorical.rename(columns = {0 : 'normal', 1 : 'Dos' ,2: 'Probe',3: 'R2L',4 :'U2R'}, inplace = True)
# numeric_data.drop('label',inplace=True, axis=1)
# numeric_data=numeric_data.join(categorical)

# numeric_data

In [29]:
 X = numeric_data.iloc[:,0:29].to_numpy() 
# #Y = numeric_data[['Dos','normal','Probe','R2L','U2R']] # target attributes

 Y = numeric_data['label']
# # splitting the dataset 75% for training and 25% testing
# X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.25, random_state=42)

In [30]:
# X = numeric_data.iloc[:,0:29] # dataset excluding target attribute (encoded, one-hot-encoded,original)
# Y = numeric_data[['Dos','normal','Probe','R2L','U2R']] # target attributes
# # splitting the dataset 75% for training and 25% testing
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.25, random_state=42)


## MODEL 1: 20,20

In [31]:
def create_model1 ():
    mlp = Sequential() # initializing model   
    # input layer and first layer with 20 neurons
    mlp.add(InputLayer(input_shape=(29, ))) # input layer
    #first layer with 20 neurons
    mlp.add(Dense(units=20, input_dim=X_train.shape[1], activation='relu'))
    mlp.add(Dense(units=20, input_dim=X_train.shape[1], activation='relu'))
    # output layer with softmax activation
    mlp.add(Dense(units=5,activation='softmax'))

    mlp.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    
# # training the model on training dataset
# history = mlp.fit(X_train, y_train, epochs=70, batch_size=5000,validation_split=0.2)

# # predicting target attribute on testing dataset
# test_results = mlp.evaluate(X_test, y_test, verbose=1)
# print(f'Test results - Loss: {test_results[0]} - Accuracy: {test_results[1]*100}%')
    return mlp

## MODEL 2: 25,25

In [32]:
def create_model2 ():
    mlp = Sequential() # initializing model
    # input layer and first layer with 93 neurons
    mlp.add(InputLayer(input_shape=(29, ))) # input layer
    #first layer with 25 neurons
    mlp.add(Dense(units=25, input_dim=X_train.shape[1], activation='relu'))
    mlp.add(Dense(units=25, input_dim=X_train.shape[1], activation='relu'))
    # output layer with softmax activation
    mlp.add(Dense(units=5,activation='softmax'))
    mlp.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    
# # training the model on training dataset
# history = mlp.fit(X_train, y_train, epochs=70, batch_size=5000,validation_split=0.2)

# # predicting target attribute on testing dataset
# test_results = mlp.evaluate(X_test, y_test, verbose=1)
# print(f'Test results - Loss: {test_results[0]} - Accuracy: {test_results[1]*100}%')
    return mlp

## MODEL 3 : DNN

In [33]:
def create_model3 ():
    model = Sequential()
    model.add(Dense(25,input_dim=29,activation='relu'))  
    model.add(Dropout(0.1))
    model.add(Dense(20,activation='relu'))  
    model.add(Dropout(0.1))
    model.add(Dense(15,activation='relu'))  
    model.add(Dropout(0.1))
    model.add(Dense(20,activation='relu'))  
    model.add(Dropout(0.1))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    return model

In [34]:
NN_clf1=KerasClassifier(build_fn=create_model1, epochs=70, batch_size= 500)
NN_clf1._estimator_type = "classifier"
NN_clf2=KerasClassifier(build_fn=create_model2, epochs=70, batch_size= 500)
NN_clf2._estimator_type = "classifier"
NN_clf3=KerasClassifier(build_fn=create_model3, epochs=70, batch_size= 500)
NN_clf3._estimator_type = "classifier"

In [35]:
intermediate = [('NN1', NN_clf1), ('NN2', NN_clf2),('NN3', NN_clf3)]

In [36]:
intermediate

[('NN1', <keras.wrappers.scikit_learn.KerasClassifier at 0x21ed334b1f0>),
 ('NN2', <keras.wrappers.scikit_learn.KerasClassifier at 0x21ed334b4f0>),
 ('NN3', <keras.wrappers.scikit_learn.KerasClassifier at 0x21ed334b8e0>)]

In [37]:
from sklearn.ensemble import StackingClassifier
from sklearn.neural_network import MLPClassifier
clf = StackingClassifier(estimators=intermediate, final_estimator=MLPClassifier())
clf.fit(X_train, y_train)

Epoch 1/70
506/506 [==============================] - 5s 5ms/step - loss: 0.4654 - accuracy: 0.8524
Epoch 2/70
506/506 [==============================] - 2s 4ms/step - loss: 0.1728 - accuracy: 0.9473
Epoch 3/70
506/506 [==============================] - 2s 4ms/step - loss: 0.1379 - accuracy: 0.9564
Epoch 4/70
506/506 [==============================] - 2s 4ms/step - loss: 0.1229 - accuracy: 0.9614
Epoch 5/70
506/506 [==============================] - 2s 5ms/step - loss: 0.1139 - accuracy: 0.9647
Epoch 6/70
506/506 [==============================] - 2s 4ms/step - loss: 0.1069 - accuracy: 0.9666
Epoch 7/70
506/506 [==============================] - 2s 4ms/step - loss: 0.1023 - accuracy: 0.9679
Epoch 8/70
506/506 [==============================] - 2s 4ms/step - loss: 0.0982 - accuracy: 0.9690
Epoch 9/70
506/506 [==============================] - 2s 4ms/step - loss: 0.0947 - accuracy: 0.9702
Epoch 10/70
506/506 [==============================] - 2s 4ms/step - loss: 0.0913 - accuracy: 0.9708

506/506 [==============================] - 4s 9ms/step - loss: -1761942568960.0000 - accuracy: 0.2003
Epoch 22/70
506/506 [==============================] - 4s 9ms/step - loss: -2115556605952.0000 - accuracy: 0.2003
Epoch 23/70
506/506 [==============================] - 4s 8ms/step - loss: -2521526697984.0000 - accuracy: 0.2003
Epoch 24/70
506/506 [==============================] - 4s 8ms/step - loss: -2979435642880.0000 - accuracy: 0.2003
Epoch 25/70
506/506 [==============================] - 4s 8ms/step - loss: -3504458170368.0000 - accuracy: 0.2003
Epoch 26/70
506/506 [==============================] - 4s 8ms/step - loss: -4093329276928.0000 - accuracy: 0.2003
Epoch 27/70
506/506 [==============================] - 4s 8ms/step - loss: -4758189899776.0000 - accuracy: 0.2003
Epoch 28/70
506/506 [==============================] - 4s 8ms/step - loss: -5496265768960.0000 - accuracy: 0.2003
Epoch 29/70
506/506 [==============================] - 4s 8ms/step - loss: -6316803227648.0000 - acc

405/405 [==============================] - 2s 4ms/step - loss: 0.0658 - accuracy: 0.9793
Epoch 36/70
405/405 [==============================] - 2s 4ms/step - loss: 0.0656 - accuracy: 0.9794
Epoch 37/70
405/405 [==============================] - 2s 4ms/step - loss: 0.0651 - accuracy: 0.9798
Epoch 38/70
405/405 [==============================] - 2s 4ms/step - loss: 0.0646 - accuracy: 0.9798
Epoch 39/70
405/405 [==============================] - 2s 4ms/step - loss: 0.0637 - accuracy: 0.9802
Epoch 40/70
405/405 [==============================] - 2s 4ms/step - loss: 0.0636 - accuracy: 0.9804
Epoch 41/70
405/405 [==============================] - 2s 4ms/step - loss: 0.0632 - accuracy: 0.9804
Epoch 42/70
405/405 [==============================] - 2s 4ms/step - loss: 0.0628 - accuracy: 0.9807
Epoch 43/70
405/405 [==============================] - 2s 4ms/step - loss: 0.0619 - accuracy: 0.9808
Epoch 44/70
405/405 [==============================] - 2s 4ms/step - loss: 0.0615 - accuracy: 0.9811
Ep

405/405 [==============================] - 1s 3ms/step - loss: 0.0571 - accuracy: 0.9823
Epoch 56/70
405/405 [==============================] - 1s 3ms/step - loss: 0.0558 - accuracy: 0.9825
Epoch 57/70
405/405 [==============================] - 1s 3ms/step - loss: 0.0556 - accuracy: 0.9825
Epoch 58/70
405/405 [==============================] - 1s 2ms/step - loss: 0.0554 - accuracy: 0.9825
Epoch 59/70
405/405 [==============================] - 1s 3ms/step - loss: 0.0548 - accuracy: 0.9827
Epoch 60/70
405/405 [==============================] - 1s 3ms/step - loss: 0.0551 - accuracy: 0.9828
Epoch 61/70
405/405 [==============================] - 1s 3ms/step - loss: 0.0543 - accuracy: 0.9830
Epoch 62/70
405/405 [==============================] - 1s 3ms/step - loss: 0.0541 - accuracy: 0.9830
Epoch 63/70
405/405 [==============================] - 1s 3ms/step - loss: 0.0544 - accuracy: 0.9830
Epoch 64/70
405/405 [==============================] - 1s 3ms/step - loss: 0.0537 - accuracy: 0.9831
Ep

405/405 [==============================] - 1s 3ms/step - loss: 0.1206 - accuracy: 0.9628
Epoch 6/70
405/405 [==============================] - 1s 3ms/step - loss: 0.1134 - accuracy: 0.9651
Epoch 7/70
405/405 [==============================] - 1s 3ms/step - loss: 0.1074 - accuracy: 0.9668
Epoch 8/70
405/405 [==============================] - 1s 3ms/step - loss: 0.1021 - accuracy: 0.9684
Epoch 9/70
405/405 [==============================] - 1s 3ms/step - loss: 0.0977 - accuracy: 0.9692
Epoch 10/70
405/405 [==============================] - 1s 3ms/step - loss: 0.0937 - accuracy: 0.9704
Epoch 11/70
405/405 [==============================] - 1s 3ms/step - loss: 0.0901 - accuracy: 0.9714
Epoch 12/70
405/405 [==============================] - 1s 3ms/step - loss: 0.0871 - accuracy: 0.9723
Epoch 13/70
405/405 [==============================] - 1s 3ms/step - loss: 0.0850 - accuracy: 0.9730
Epoch 14/70
405/405 [==============================] - 1s 3ms/step - loss: 0.0823 - accuracy: 0.9740
Epoch 

405/405 [==============================] - 1s 3ms/step - loss: 0.0707 - accuracy: 0.9774
Epoch 26/70
405/405 [==============================] - 1s 3ms/step - loss: 0.0699 - accuracy: 0.9775
Epoch 27/70
405/405 [==============================] - 1s 3ms/step - loss: 0.0692 - accuracy: 0.9778
Epoch 28/70
405/405 [==============================] - 1s 3ms/step - loss: 0.0678 - accuracy: 0.9781
Epoch 29/70
405/405 [==============================] - 1s 3ms/step - loss: 0.0671 - accuracy: 0.9784
Epoch 30/70
405/405 [==============================] - 1s 3ms/step - loss: 0.0670 - accuracy: 0.9782
Epoch 31/70
405/405 [==============================] - 1s 3ms/step - loss: 0.0655 - accuracy: 0.9786
Epoch 32/70
405/405 [==============================] - 1s 3ms/step - loss: 0.0650 - accuracy: 0.9788
Epoch 33/70
405/405 [==============================] - 1s 3ms/step - loss: 0.0641 - accuracy: 0.9791
Epoch 34/70
405/405 [==============================] - 1s 3ms/step - loss: 0.0640 - accuracy: 0.9791
Ep

405/405 [==============================] - 3s 8ms/step - loss: -5830544982016.0000 - accuracy: 0.2003
Epoch 42/70
405/405 [==============================] - 3s 8ms/step - loss: -6411650596864.0000 - accuracy: 0.2003
Epoch 43/70
405/405 [==============================] - 3s 8ms/step - loss: -7064585764864.0000 - accuracy: 0.2003
Epoch 44/70
405/405 [==============================] - 3s 8ms/step - loss: -7745212252160.0000 - accuracy: 0.2003
Epoch 45/70
405/405 [==============================] - 3s 8ms/step - loss: -8480373080064.0000 - accuracy: 0.2003
Epoch 46/70
405/405 [==============================] - 3s 8ms/step - loss: -9261686980608.0000 - accuracy: 0.2003
Epoch 47/70
405/405 [==============================] - 3s 8ms/step - loss: -10116488232960.0000 - accuracy: 0.2003
Epoch 48/70
405/405 [==============================] - 3s 8ms/step - loss: -11034997817344.0000 - accuracy: 0.2003
Epoch 49/70
405/405 [==============================] - 3s 8ms/step - loss: -11995217657856.0000 - 

405/405 [==============================] - 2s 5ms/step - loss: -8550595166208.0000 - accuracy: 0.2003
Epoch 44/70
405/405 [==============================] - 2s 4ms/step - loss: -9378283388928.0000 - accuracy: 0.2003
Epoch 45/70
405/405 [==============================] - 2s 4ms/step - loss: -10264763170816.0000 - accuracy: 0.2003
Epoch 46/70
405/405 [==============================] - 2s 5ms/step - loss: -11204385832960.0000 - accuracy: 0.2003
Epoch 47/70
405/405 [==============================] - 2s 5ms/step - loss: -12217897451520.0000 - accuracy: 0.2003
Epoch 48/70
405/405 [==============================] - 2s 4ms/step - loss: -13296904175616.0000 - accuracy: 0.2003
Epoch 49/70
405/405 [==============================] - 2s 4ms/step - loss: -14476157911040.0000 - accuracy: 0.2003
Epoch 50/70
405/405 [==============================] - 2s 4ms/step - loss: -15728134586368.0000 - accuracy: 0.2003
Epoch 51/70
405/405 [==============================] - 2s 5ms/step - loss: -17005227802624.000

405/405 [==============================] - 2s 5ms/step - loss: -12007242727424.0000 - accuracy: 0.2003
Epoch 46/70
405/405 [==============================] - 2s 5ms/step - loss: -13114108018688.0000 - accuracy: 0.2003
Epoch 47/70
405/405 [==============================] - 2s 5ms/step - loss: -14320711761920.0000 - accuracy: 0.2003
Epoch 48/70
405/405 [==============================] - 2s 5ms/step - loss: -15623641890816.0000 - accuracy: 0.2003
Epoch 49/70
405/405 [==============================] - 2s 5ms/step - loss: -16974025326592.0000 - accuracy: 0.2003
Epoch 50/70
405/405 [==============================] - 2s 5ms/step - loss: -18438007291904.0000 - accuracy: 0.2003
Epoch 51/70
405/405 [==============================] - 2s 5ms/step - loss: -19960917131264.0000 - accuracy: 0.2003
Epoch 52/70
405/405 [==============================] - 2s 5ms/step - loss: -21627481882624.0000 - accuracy: 0.2003
Epoch 53/70
405/405 [==============================] - 2s 5ms/step - loss: -23397037768704.0

StackingClassifier(estimators=[('NN1',
                                <keras.wrappers.scikit_learn.KerasClassifier object at 0x0000021ED334B1F0>),
                               ('NN2',
                                <keras.wrappers.scikit_learn.KerasClassifier object at 0x0000021ED334B4F0>),
                               ('NN3',
                                <keras.wrappers.scikit_learn.KerasClassifier object at 0x0000021ED334B8E0>)],
                   final_estimator=MLPClassifier())

In [38]:
print("Stacking model score: %.7f" % clf.score(X_test, y_test))
# clf.predict_proba(X_test)

2631/2631 [==============================] - 3s 1ms/step
Stacking model score: 0.9894748


In [39]:
 pred = clf.predict(X_test)

2631/2631 [==============================] - 3s 1ms/step


In [25]:
y_test

378       4
32038     0
86399     0
74412     0
52951     1
         ..
100955    4
77744     4
17242     4
118703    1
119625    0
Name: intrusion_no, Length: 31494, dtype: int64

In [22]:
pred

array([4, 0, 0, ..., 4, 1, 0], dtype=int64)

In [40]:
pred = clf.predict_proba(X_test)
print("Log Loss - ",log_loss(y_test,pred))

2631/2631 [==============================] - 3s 1ms/step
Log Loss -  0.040135660910999515


In [43]:
pred = clf.predict(X_test)
accuracy = accuracy_score(y_test,pred)
print('Accuracy: %8f' % accuracy)

precision = precision_score(y_test,pred,average='macro')
print('Precision: %8f' % precision)

recall = recall_score(y_test,pred,average='macro')
print('Recall: %8f' % recall)

f1 = f1_score(y_test,pred,average='macro')
print('F1 score: %8f' % f1)

2631/2631 [==============================] - 3s 1ms/step
Accuracy: 0.989475
Precision: 0.989509
Recall: 0.989514
F1 score: 0.989494
